# Search Supplier

This program compares two lists of companies and identify possible matches in both lists


In [1]:
##############################################################
# Configuration values
##############################################################

import os

## Data Folder and File Names
DATA_DIR                = './data'


SCBN_SUPPLIER_FILE_PATH  = os.path.join(DATA_DIR, 'GIODEUsageReportCOMBINED03092020.xlsx')
VAN_SUPPLIER_FILE_PATH   = os.path.join(DATA_DIR, 'Trading partners using the 846 document.xlsx')
CDC_SUPPLIER_INDEX_PAGE = 'https://www.cdc.gov/niosh/npptl/topics/respirators/disp_part/N95list1.html'

REPORT_FILE_PATH         = os.path.join(DATA_DIR, 'supplier_match_result.xlsx')

NAME_MATCH_TOCKEN_LENGTH = 5


In [2]:

import numpy as np
import pandas as pd
import string
from urllib.error import URLError, HTTPError
import cdc_parser as cdcprsr
import re

def read_scbn_supplier_list(filepath: str, sheetnum: int, headerrownum: int) -> pd.DataFrame:
    # load SCBN supplier list

    scbn_suppliers_all = pd.read_excel(filepath,
            sheetnum, # Read the first worksheet
            headerrownum, # Header is on row 0 (row 1 in Excel's 1base count)
            )

    scbn_suppliers_846 = scbn_suppliers_all[scbn_suppliers_all['DOCUMENTTYPENAME'] == 846]

    # print(scbn_suppliers_846)

    return_list = scbn_suppliers_846[['NAME', 'BUSINESSALIASNAME', 'NUMBEROFDOCS']]

    return_list.columns = ['name', 'alias', 'doc_count']

    return_list['system'] = 'SCBN'

    return return_list

def read_van_supplier_list(filepath: str, sheetnum: int, headerrownum: int) -> pd.DataFrame:
    # load SCBN supplier list

    van_suppliers_all = pd.read_excel(filepath,
            sheetnum, # Read the first worksheet
            headerrownum, # Header is on row 0 (row 1 in Excel's 1base count)
            )

    return_list = van_suppliers_all[['Company Name', 'Sender ID', 'Doc Count']]

    return_list.columns = ['name', 'alias', 'doc_count']

    return_list['system'] = 'VAN'

    return return_list

In [3]:

def get_cdc_supplier_list (cdc_supplier_index_url: str) -> pd.DataFrame:

    # Read CDC Approved Respirator company list

    # get the links to all of the pages of CDC approved supplier from the index page
    requestlist = cdcprsr.get_cdc_supplier_page_list(cdc_supplier_index_url)

    cdc_supplier_results = []

    for req in requestlist:

        try:
            html_tables = pd.read_html(
                req,
                attrs = { 'class': 'table'},
                match = 'Manufacturers Listed Alphabetically'
                )
        except HTTPError as httperr:
            print("Cannot load the page of {} with the exception of {}".format(url, httperr))
            html_tables = None
        

        if html_tables is not None and len(html_tables) > 0:

            supplier_on_page = html_tables[0]
            cdc_supplier_results.append(supplier_on_page)


    cdc_supplier_list = pd.concat(
        cdc_supplier_results,
        ignore_index=True,
    )

    cdc_supplier_list['name'] = cdc_supplier_list['Supplier/Manufacturer and Contact Information'].str.split('external icon').str[0]

    return cdc_supplier_list




In [4]:
def construct_matchvalue(inputrow, descfieldname, numoftokens):

    TOKEN_IGNORE_START_WITH = ('incorp', 'compan', 'corp', 'ltd', 'llp', )
    TOKEN_IGNORE = ['inc', 'co', 'prod']

    inputname = inputrow[descfieldname]

    if isinstance(inputname, str):
        # split the name into tokens by space
        name_tokens = inputname.lower().split()

        match_tokens = []

        for ele in name_tokens:
            # only use tokens with letters (including unicode letters) in them
            if re.search('[^\d\W]+', ele):
                # keep only alphnumeric character in a token
                ele_only_alpha = re.sub('[\W]', '', ele)

                if ele_only_alpha is not None and len(ele_only_alpha) > 0 \
                and not ele_only_alpha.startswith(TOKEN_IGNORE_START_WITH) \
                and not ele_only_alpha in TOKEN_IGNORE:
                    match_tokens.append(ele_only_alpha)

        return ' '.join(match_tokens[0:numoftokens])


    else:
        return 'NO DESCRIPTION!!!'

In [5]:
cdc_suppliers = get_cdc_supplier_list(CDC_SUPPLIER_INDEX_PAGE)

# Constructed the MatchValue column using the construct_matchvalue function
cdc_suppliers['name2match'] = cdc_suppliers.apply(
    construct_matchvalue, 
    args=('name', NAME_MATCH_TOCKEN_LENGTH), 
    axis=1,
    )
    

In [6]:
# get scbn suppliers
scbn_suppliers = read_scbn_supplier_list(SCBN_SUPPLIER_FILE_PATH, 0, 0)

# scbn_suppliers

In [7]:
van_suppliers = read_van_supplier_list(VAN_SUPPLIER_FILE_PATH, 0, 0)

# van_suppliers

In [8]:
ibm_suppliers = pd.concat(
    [scbn_suppliers, van_suppliers],
    ignore_index=True,
    keys='name',
    )

# ibm_suppliers.reindex()

ibm_suppliers['name'] = ibm_suppliers['name'].str.upper()

# Constructed the MatchValue column using the construct_matchvalue function
ibm_suppliers['name2match'] = ibm_suppliers.apply(
    construct_matchvalue, 
    args=('name', NAME_MATCH_TOCKEN_LENGTH), 
    axis=1,
    )

# ibm_suppliers

In [9]:

ibm_suppliers['scbn_van_transactions'] = ibm_suppliers['system'] + '(' + ibm_suppliers['doc_count'].astype(str) + ')'

ibm_supplier_agg = ibm_suppliers.groupby(['name2match'])['scbn_van_transactions'].apply(', '.join)

# ibm_supplier_agg.set_index(['name', 'nametomatch'])
# ibm_supplier_agg


In [10]:
# cdc_suppliers.reindex()

cdc_supplier_agg = cdc_suppliers.groupby(['name', 'name2match'])['Approval Number'].apply(', '.join)

# cdc_supplier_agg.set_index(['name', 'nametomatch'])

# cdc_supplier_agg

In [13]:

ibm_supplier_agg.reset_index()

match_result = cdc_supplier_agg.reset_index().merge(
    ibm_supplier_agg,
    on=['name2match'],
    # left_on='name2match',
    # right_on='name2match',
    how='left',
    suffixes=('_CDC', '_IBM'),
    # indicator=True,
)

match_result.columns = ['company name', 'match name', 'cdc N95 model approval number', 'ibm network 846 transactions']

match_result

,company name,match name,cdc N95 model approval number,ibm network 846 transactions
0,3M Company,3m,"84A-0006, 84A-0007, 84A-0008, 84A-0454, 84A-12...",VAN(38.0)
1,4522915 Canada Inc,canada,"84A-3888, 84A-3889",NaN
2,A.R. Medicom Inc. (ASIA) Ltd.,ar medicom asia,"84A-3323, 84A-4541, 84A-7643",NaN
3,"A.R. Medicom Inc., Ltd.",ar medicom,"84A-4006, 84A-4282",NaN
4,AARON Industrial Safety,aaron industrial safety,84A-0160,NaN
...,...,...,...,...
345,Zee Medical,zee medical,84A-0328,NaN
346,"Zee Medical, Inc.",zee medical,"84A-0160, 84A-0161",NaN
347,"Zhou Medical Solutions, LLC",zhou medical solutions llc,"84A-5411, 84A-5463",NaN
348,i-Nano Industries Pte.,inano industries pte,84A-5037,NaN


In [14]:
# persist cdc approved supplier list and ibm supplier list to an Excel

output_writer = pd.ExcelWriter(REPORT_FILE_PATH)

cdc_suppliers.to_excel(output_writer, sheet_name='cdc approved suppliers')
cdc_supplier_agg.to_excel(output_writer, sheet_name='cdc suppliers agg')

ibm_suppliers.to_excel(output_writer, sheet_name='suppliers on ibm network')
ibm_supplier_agg.to_excel(output_writer, sheet_name='suppliers on ibm agg')

match_result.to_excel(output_writer, sheet_name='match result')

output_writer.save()
output_writer.close()